##Step 1 : Import Module

In [1]:
import numpy as np
from enum import Enum, auto

from IPython.core.display import HTML,display

import pandas as pd
from datetime import datetime

##Step 2 : Deklarasi Variabel

In [20]:
# String Variable
sequenceA       : str = 'FEEDS'#'TREES'
sequenceB       : str = 'TREES'#'FEEDS'
alignedA        : str
alignedB        : str

# Integer Variable
lengthSequenceA : int
lengthSequenceB : int

limit = 999999

# Matrix
main_matrix           : list
match_checker_matrix  : list

In [21]:
class Sequences(Enum) :
  MATCH: int = 1
  MISS_MATCH = -1
  GAP_PENALTY = -1

class Arrow(Enum) :
  UP          : str = "\u2191"
  RIGHT       : str = "\u2192"
  down_arrow  : str = "\u2193"
  LEFT        : str = "\u2190"
  DOWN_RIGHT  : str = "\u2198"
  UP_LEFT     : str = "\u2196"

##Step 3 : Method Function

In [7]:
# Fungsi generate data table
def pretty_table_from_array(data_array, row_labels,col_labels):
    """Show an HTML table from a 2d numpy array"""
    df = pd.DataFrame(data_array,index=row_labels,columns=col_labels)
    table_html = df.to_html()
    return HTML(table_html)

In [8]:
def traceback_alignment(traceback_array, sequenceA, sequenceB):

    n_rows = len(sequenceA) + 1 #need an extra row up top
    n_columns = len(sequenceB) + 1 #need an extra row up top

    row = len(sequenceA)
    col = len(sequenceB)
    arrow = traceback_array[row,col]
    aligned_sequenceA = ""
    aligned_sequenceB = ""
    alignment_indicator = ""
    while arrow is not "-":
            # print("Currently on row:",row)
            # print("Currently on col:",col)
            arrow = traceback_array[row,col]
            # print("Arrow:",arrow)

            if arrow == Arrow.UP.value:
                # print("insert indel into top sequence")
                #We want to add the new indel onto the left
                #side of the growing aligned sequence
                aligned_sequenceB = "-"+aligned_sequenceB
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                alignment_indicator = " "+alignment_indicator
                row -=1

            elif arrow == Arrow.UP_LEFT.value:
                # print("match or mismatch")
                #Note that we look up the row-1 and col-1 indexes
                #because there is an extra "-" character at the
                #start of each sequence
                sequenceA_character = sequenceA[row-1]
                sequenceB_character = sequenceB[col-1]
                aligned_sequenceA = sequenceA[row-1] + aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                if sequenceA_character == sequenceB_character:
                    alignment_indicator = "|"+alignment_indicator
                else:
                    alignment_indicator = " "+alignment_indicator
                row -=1
                col -=1

            elif arrow == Arrow.LEFT.value:
                # print("Insert indel into left sequence")
                aligned_sequenceA = "-"+aligned_sequenceA
                aligned_sequenceB = sequenceB[col-1] + aligned_sequenceB
                alignment_indicator = " "+alignment_indicator
                col -=1

            elif arrow == "-":
                break
            else:
                raise ValueError(f"Traceback array entry at {row},{col}: {arrow} is not recognized as an up arrow ({Arrow.UP.value}),Arrow.LEFT.value ({Arrow.LEFT.value}), Arrow.UP_LEFT.value ({Arrow.UP_LEFT.value}), or a stop (-).")
            #print(traceback_array,-row,-col,traceback_array[-row,-col])
    print(aligned_sequenceA)
    print(alignment_indicator)
    print(aligned_sequenceB)

    # return aligned_sequenceA,aligned_sequenceB

<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-8-64f97e14f2b3>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  while arrow is not "-":
<ipython-input-8-64f97e14f2b3>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  while arrow is not "-":


##Step 4 : Matrix initiate

In [9]:
start = datetime.now()
baris = len(sequenceA) + 1
kolom = len(sequenceB) + 1

main_matrix           = np.full([baris, kolom], 0)
match_checker_matrix  = np.full([baris, kolom], '-')

row_labels = [label for label in "-"+sequenceA]
column_labels = [label for label in "-"+sequenceB]

# print("Scoring array:")
# display(pretty_table_from_array(main_matrix,row_labels,column_labels))
# print("\n\nTraceback array:")
# display(pretty_table_from_array(match_checker_matrix,row_labels,column_labels))
print("time :: " + str(datetime.now() - start))

time :: 0:00:00.000563


##Step 5 : Needleman Wunsch Algorithm

### Step 1 : Initialisation

In [16]:
# Matriks Score
for i in range(0, len(main_matrix)):
  if i == 0:
    for j in range(0, len(main_matrix[i])):
      main_matrix[i][j] = j * Sequences.GAP_PENALTY.value
  else:
    main_matrix[i][0] = i * Sequences.GAP_PENALTY.value

# print("Scoring array:")
# display(pretty_table_from_array(main_matrix,row_labels,column_labels))

In [17]:
#Matriks Traceback
for i in range(0, len(main_matrix)):
  if i == 0:
    for j in range(0, len(main_matrix[i])):
      if j == 0:
        match_checker_matrix[i][j] = "-"
      else:
        match_checker_matrix[i][j] = Arrow.LEFT.value
  else:
    match_checker_matrix[i][0] = Arrow.UP.value

# print("Scoring array:")
# display(pretty_table_from_array(match_checker_matrix,row_labels,column_labels))

### Step 2 : Matrix Filling

In [18]:
start = datetime.now()
for i in range(1,len(sequenceB)+1):
    for j in range(1,len(sequenceA)+1):
      cell_to_the_left = main_matrix[i][j-1]
      from_left_score = cell_to_the_left + Sequences.GAP_PENALTY.value

      above_cell = main_matrix[i-1][j]
      from_above_score = above_cell + Sequences.GAP_PENALTY.value

      diagonal_left_cell = main_matrix[i-1][j-1]

      if sequenceB[i-1] == sequenceA[j-1]:
          diagonal_left_cell_score = diagonal_left_cell + Sequences.MATCH.value
      else:
          diagonal_left_cell_score = diagonal_left_cell + Sequences.MISS_MATCH.value

      score = max([from_left_score,from_above_score,diagonal_left_cell_score])

      main_matrix[i][j] = score

      if score == from_left_score:
          arrow = Arrow.LEFT.value
      elif score == from_above_score:
          arrow = Arrow.UP.value
      elif score == diagonal_left_cell_score:
          arrow = Arrow.UP_LEFT.value
      match_checker_matrix[i][j] = arrow

print("Scoring array:")
display(pretty_table_from_array(main_matrix,row_labels,column_labels))
print("\n\nTraceback array:")
display(pretty_table_from_array(match_checker_matrix,row_labels,column_labels))

print("time :: " + str(datetime.now() - start))

# print(main_matrix)

Scoring array:


,-,T,R,E,E,S
-,0,-1,-2,-3,-4,-5
F,-1,-1,-2,-3,-4,-5
E,-2,-2,-2,-3,-4,-5
E,-3,-3,-1,-1,-2,-3
D,-4,-4,-2,0,-1,-2
S,-5,-5,-3,-1,-1,0




Traceback array:


,-,T,R,E,E,S
-,-,←,←,←,←,←
F,↑,↖,←,←,←,←
E,↑,↑,↖,←,←,←
E,↑,↑,↖,↖,←,←
D,↑,↑,↑,↖,←,←
S,↑,↑,↑,↑,↖,↖


time :: 0:00:00.022970


### Step 3 : Traceback

In [19]:
# Algoritma Needleman Wunsch
traceback_alignment(match_checker_matrix, sequenceB, sequenceA)

TREE-S
  || |
F-EEDS
